In [0]:
#trades Example Input:
order_id    user_id    price    quantity    status    timestamp
100101        111        9.80    10            Cancelled    08/17/2022 12:00:00
100102        111        10.00    10            Completed    08/17/2022 12:00:00
100259        148        5.10    35            Completed    08/25/2022 12:00:00
100264        148        4.80    40            Completed    08/26/2022 12:00:00
100305        300        10.00    15            Completed    09/05/2022 12:00:00
100400        178        9.90    15            Completed    09/09/2022 12:00:00
100565        265        25.60    5            Completed    12/19/2022 12:00:00


In [0]:
#users Example Input:
user_id    city            email                    signup_date
111        San Francisco    rrok10@gmail.com        08/03/2021 12:00:00
148        Boston            sailor9820@gmail.com    08/20/2021 12:00:00
178        San Francisco    harrypotterfan182@gmail.com    01/05/2022 12:00:00
265        Denver            shadower_@hotmail.com    02/26/2022 12:00:00
300        San Francisco    houstoncowboy1122@hotmail.com    06/30/2022 12:00:00

In [0]:
Assume you're given the tables containing completed trade orders and user details in a Robinhood trading system.
Write a query to retrieve the top three cities that have the highest number of completed trade orders listed in descending order. Output the city name and the corresponding number of completed trade orders.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
from pyspark.sql.functions import *

In [0]:
%sql

CREATE TABLE trades (
order_id INT,
user_id INT,
price FLOAT,
quantity INT,
status VARCHAR(20),
timestamps TIMESTAMP
);

In [0]:
%sql
INSERT INTO trades (order_id, user_id, price, quantity, status, timestamps) VALUES
(100101, 111, 9.80, 10, 'Cancelled', '2022-08-17 12:00:00'),
(100102, 111, 10.00, 10, 'Completed', '2022-08-17 12:00:00'),
(100259, 148, 5.10, 35, 'Completed', '2022-08-25 12:00:00'),
(100264, 148, 4.80, 40, 'Completed', '2022-08-26 12:00:00'),
(100305, 300, 10.00, 15, 'Completed', '2022-09-05 12:00:00'),
(100400, 178, 9.90, 15, 'Completed', '2022-09-09 12:00:00'),
(100565, 265, 25.60, 5, 'Completed', '2022-12-19 12:00:00');

num_affected_rows,num_inserted_rows
7,7


In [0]:
%sql
CREATE TABLE users (
user_id INT,
city VARCHAR(50),
email VARCHAR(50),
signup_date TIMESTAMP
);

In [0]:
%sql
INSERT INTO users (user_id, city, email, signup_date) VALUES
(111, 'San Francisco', 'rrok10@gmail.com', '2021-08-03 12:00:00'),
(148, 'Boston', 'sailor9820@gmail.com', '2021-08-20 12:00:00'),
(178, 'San Francisco', 'harrypotterfan182@gmail.com', '2022-01-05 12:00:00'),
(265, 'Denver', 'shadower_@hotmail.com', '2022-02-26 12:00:00'),
(300, 'San Francisco', 'houstoncowboy1122@hotmail.com', '2022-06-30 12:00:00');

num_affected_rows,num_inserted_rows
5,5


In [0]:
trades_df = spark.read.table("trades")
trades_df.display()

order_id,user_id,price,quantity,status,timestamps
100101,111,9.8,10,Cancelled,2022-08-17T12:00:00.000+0000
100102,111,10.0,10,Completed,2022-08-17T12:00:00.000+0000
100259,148,5.1,35,Completed,2022-08-25T12:00:00.000+0000
100264,148,4.8,40,Completed,2022-08-26T12:00:00.000+0000
100305,300,10.0,15,Completed,2022-09-05T12:00:00.000+0000
100400,178,9.9,15,Completed,2022-09-09T12:00:00.000+0000
100565,265,25.6,5,Completed,2022-12-19T12:00:00.000+0000


In [0]:
users_df = spark.read.table("users")
users_df.display()

user_id,city,email,signup_date
111,San Francisco,rrok10@gmail.com,2021-08-03T12:00:00.000+0000
148,Boston,sailor9820@gmail.com,2021-08-20T12:00:00.000+0000
178,San Francisco,harrypotterfan182@gmail.com,2022-01-05T12:00:00.000+0000
265,Denver,shadower_@hotmail.com,2022-02-26T12:00:00.000+0000
300,San Francisco,houstoncowboy1122@hotmail.com,2022-06-30T12:00:00.000+0000


In [0]:
%sql
--#Write a query to retrieve the top three cities that have the highest number of completed trade orders listed in descending order. Output the city name and the corresponding number of completed trade orders.

SELECT u.city, count(*) as num_completed_trades
from trades as t
join users as u
ON t.user_id = u.user_id
where status='Completed'
GROUP BY u.city
ORDER BY num_completed_trades DESC
LIMIT 3;

city,num_completed_trades
San Francisco,3
Boston,2
Denver,1


In [0]:
#Write a query to retrieve the top three cities that have the highest number of completed trade orders listed in descending order. Output the city name and the corresponding number of completed trade orders.

join_df = trades_df.join(users_df, trades_df.user_id == users_df.user_id)
filter_df = join_df.filter(join_df.status == "Completed")
count_df = filter_df.groupBy("city").agg(count("*").alias("num_completed_trades"))
order_df = count_df.sort(desc("num_completed_trades"))

result_df = order_df.limit(3)
result_df.show()

+-------------+--------------------+
|         city|num_completed_trades|
+-------------+--------------------+
|San Francisco|                   3|
|       Boston|                   2|
|       Denver|                   1|
+-------------+--------------------+

